## 18. Application: DICOM

DICOM (Digital Imaging and Communications in Medicine) is the international standard to transmit, store, retrieve, print, process, and display medical imaging information. It is in particular widely used to store volumetric data from methods such as CT, MR, Ultrasound, etc.

This kind of specific image format is typically not supported by general packages such as scikit-image. However in most cases, independent dedicated packages exist. A simple Google search leads us to the [pydicom](https://pydicom.github.io/pydicom/stable/getting_started.html) package.

In [70]:
import os
import matplotlib.pyplot as plt
plt.gray()
import pydicom
import numpy as np
import skimage

<Figure size 432x288 with 0 Axes>

We will use an MRI dataset of a head available on the data sharing platform Zenodo. In this course, most data have been made directly available. To show the full procedure, we will here include the download step.

## 18.1. Download

The donwload address on Zenodo is:

In [72]:
data_address= 'https://zenodo.org/record/16956/files/DICOM.zip?download=1'

We can use the ```urllib``` native package to proceed with download which provides us with a zip file:

In [73]:
import urllib

urllib.request.urlretrieve(data_address, 'Data/mri.zip')

('Data/mri.zip', <http.client.HTTPMessage at 0x1480b7a50>)

To automate the process we now also automatically unzip the file using the ```zipfile``` module:

In [75]:
import zipfile

In [76]:
with zipfile.ZipFile('Data/mri.zip', 'r') as zip_ref:
    zip_ref.extractall('Data/mri/')

## 18.2. Importing one slice

We define the general path to the folder containing slices:

In [77]:
path = '/Users/gw18g940/Desktop/temp/DICOM/ST000000/SE000002/'

Now we use the ```pydicom``` package to import a single slice using the ```dcmread()``` function:

In [79]:
single_slice = pydicom.dcmread(path+'MR000000')

A DICOM file does not just contain image data but a very extensive set of metadata. You can see these metadata by just printing the variable:

In [85]:
single_slice;

All that information is also available as attributes of the variable. For example you can get the patient's name:

In [86]:
single_slice.PatientName

'LIONHEART^WILLIAM'

But also numerical values such as pixel spacing or position of slice in the stack:

In [93]:
single_slice.PixelSpacing

[0.8984375, 0.8984375]

In [94]:
single_slice.SliceLocation

"0.0"

In [95]:
test = itk.imread(path+'MR000001')
scaling = test.GetSpacing()

## 18.3. Loading the complete stack

As we have already done previously, we have first to parse the folder content to gather the files belonging to the stack. Here we simply list the folder content:

In [111]:
file_list = os.listdir(path)

In [112]:
file_list

['MR000000',
 'MR000007',
 'MR000031',
 'MR000009',
 'MR000008',
 'MR000030',
 'MR000006',
 'MR000001',
 'MR000023',
 'MR000024',
 'MR000012',
 'MR000015',
 'MR000014',
 'MR000013',
 'MR000025',
 'MR000022',
 'MR000004',
 'MR000003',
 'MR000002',
 'MR000005',
 'MR000027',
 'MR000018',
 'MR000020',
 'MR000016',
 'MR000029',
 'MR000011',
 'MR000010',
 'MR000017',
 'MR000028',
 'MR000021',
 'MR000026',
 'MR000019']

We can now load each slice using a comprehension list. From the file sorting, we already see that we'll later have to reorder the slices.

In [113]:
slices = [pydicom.dcmread(path+x) for x in os.listdir(path)]

In principle we could reorder the file by names but this is going to depend on file name formatting. A more general solution is to reorganize based on the location of the file in the stack. Let's recover that position:

In [114]:
positions = [int(x.SliceLocation) for x in slices]

In [116]:
positions

[0,
 42,
 185,
 54,
 48,
 180,
 36,
 5,
 137,
 144,
 72,
 90,
 84,
 78,
 149,
 132,
 23,
 18,
 12,
 30,
 161,
 108,
 120,
 96,
 174,
 66,
 60,
 102,
 168,
 126,
 156,
 114]

We then use ```np.argsort()``` function to get the indices of the ordered list:

In [118]:
index_ordered = np.argsort(positions)

In [119]:
index_ordered

array([ 0,  7, 18, 17, 16, 19,  6,  1,  4,  3, 26, 25, 10, 13, 12, 11, 23,
       27, 21, 31, 22, 29, 15,  8,  9, 14, 30, 20, 28, 24,  5,  2])

And finally use that ordered list to reorder the slices themselves:

In [121]:
reordered = []
slices_ordered = [slices[x] for x in index_ordered]

## 18.4. Visualization

Finally we can visualize our volume. First let's create an actual volume by stacking the planes:

In [123]:
volume = np.stack([x.pixel_array for x in slices_ordered])

In [124]:
volume.shape

(32, 256, 256)

For the rendering, we'll see here two different solutions. The first one is ipyvolume, a leight-weight volume viewer purely based on browser technology. The syntax is very similar to matplotlib.

In [125]:
import ipyvolume as ipv

In [132]:
ipv.figure()
ipv.volshow(volume)
ipv.show()

As ipyvolume is fully browser-based, it's very easy to save an image as a web page. For example we can just type:

In [133]:
ipv.save('test.html')

And this saves for us a full interactive version of the figure above. This can therefore be very useful for demonstration purposed e.g. to insert an image on a web-page.

Note that customizing the aspect of the view requires some work and that this package is not as mature as others.

An alternative solution is to use the ITK (Insight Toolkit), a very popular image processing tool suite in medical imaging (an interesting but more challenging alternative to scikit-image). ITK in particular offers a volume viewer compatible with Python and Jupyter:

In [136]:
import itkwidgets as itkw
import itk

We can just call the ```view()``` function:

In [137]:
itkw.view(volume)

Viewer(geometries=[], gradient_opacity=0.22, point_sets=[], rendered_image=<itkImagePython.itkImageUS3; proxy …

We see that the head looks compressed because the acquisition is anisotropic (large depth dimension that width/height). Above we simply passed a Numpy array to the viewer. However we can also create a native ITK format to adjust parameters more easily:

In [138]:
image_from_array = itk.image_from_array(volume)

This object has now several new attributes and methods such as:

In [139]:
image_from_array.GetSpacing()

itkVectorD3 ([1, 1, 1])

We can try to guess and adjust the spacing:

In [155]:
image_from_array.SetSpacing((1,1,10))

In [157]:
itkw.view(image_from_array)

Viewer(geometries=[], gradient_opacity=0.22, point_sets=[], rendered_image=<itkImagePython.itkImageUS3; proxy …

## 18.5. Image processing

Finally, we can do the same image processing operations as we did before, just in 3D. For example a thresholding:

In [158]:
import skimage.filters

In [170]:
vol_thresh = volume>200

In [169]:
itkw.view(vol_thresh.astype(np.uint8))

Viewer(geometries=[], gradient_opacity=0.22, point_sets=[], rendered_image=<itkImagePython.itkImageUC3; proxy …